In [1]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 8it [00:41,  3.44s/it]                       

In [10]:
from openpyxl import Workbook
from datetime import datetime
from utils import min_to_jour

def show_solution(model, OCCUPATIONS):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active
    sillons_df = dataframes["sillons_df"]
    taches_df = dataframes["taches_df"]
    machines_df = dataframes["machines_df"]

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "Type de tache"
    sheet["C1"] = "Jour"
    sheet["D1"] = "HDébut"
    sheet["E1"] = "Durée"
    sheet["F1"] = "Sillon"
    sheet["G1"] = "Minute"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            sheet[f"B{row}"] = tache
            sheet[f"A{row}"] = tache + "_" + sillon_id
            sheet[f"C{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[0]
            sheet[f"D{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[1]
            if tache in list(machines_df["Machine"]):
                sheet[f"E{row}"] = machines_df[machines_df["Machine"] == tache]["Duree "].iloc[0]
            elif tache in list(taches_df["Type de tache humaine"]): 
                sheet[f"E{row}"] = taches_df[taches_df["Type de tache humaine"] == tache]["Durée"].iloc[0]
            sheet[f"F{row}"] = sillon_id
            sheet[f"G{row}"] = var_dict[tache][sillon_id].X
            row += 1
    ##ocupation
    L, occupation_limits = retrieve_occupations(OCCUPATIONS)
    occupation_sheet = workbook.create_sheet("Occupation")
    sheet = occupation_sheet
    sheet["A1"] = "Mesure"
    sheet["B1"] = "WPY_REC"
    sheet["C1"] = "WPY_FOR"
    sheet["D1"] = "WPY_DEP"
    
    sheet["A2"] = "Taux max occupation"
    sheet["A3"] = "Nb max occupation"
    sheet["A4"] = "Nb max par chantier"
    
    chantier = "WPY_REC"
    occupation_max = max(L[0])
    capacity = occupation_limits[0]
    sheet["B2"] = 100*occupation_max/capacity
    sheet["B3"] = occupation_max
    sheet["B4"] = capacity
    
    chantier = "WPY_FOR"
    occupation_max = max(L[1])
    capacity = occupation_limits[1]
    sheet["C2"] = 100*occupation_max/capacity
    sheet["C3"] = occupation_max
    sheet["C4"] = capacity
    
    chantier = "WPY_DEP"
    occupation_max = max(L[2])
    capacity = occupation_limits[2]
    sheet["D2"] = 100*occupation_max/capacity
    sheet["D3"] = occupation_max
    sheet["D4"] = capacity
    
    currentDateAndTime = datetime.now().strftime("%d-%m_%H-%M-%S")
    workbook.save(filename=f"results/{data.NAME}-{currentDateAndTime}.xlsx")
# show_solution(m)


In [3]:
import variables
import contraintes
import importlib
import utils
importlib.reload(variables) #to remive in final .py

importlib.reload(utils)

<module 'utils' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\utils.py'>

In [4]:
# create model
importlib.reload(contraintes) #to remive in final .py
m = Model()

# generate variables (integers)
var_dict = variables.generate_variablesJ1(m, dataframes)
# generate variables
B, OCCUPATIONS = contraintes.generate_contraintes(m, dataframes, var_dict)

final_train_task = dataframes["sillons_df"].copy()
final_train_task = final_train_task[final_train_task["LDEP"]=="WPY"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
# final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
# final_train_task = var_dict["essai de frein départ"][final_train_task]
final_train_task = quicksum([var_dict["essai de frein départ"][temp] for temp in list(final_train_task["train_id"])])
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)
# autre idée : maximiser la moyenne de l'ecart entre frein de depart et HDEP
# aute idee minimiser occupation
m.update()
# m.printStats()
# m.write("genconstr.mps")
# m.write("test.lp")
m.optimize()
# m.write("test.lp")

show_solution(m, OCCUPATIONS)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-26


Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 793.43it/s]


c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:195: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:204: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  list_wagons = list(correspondances_df[correspondances_df["train_id"] ==
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:207: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
BUILDING CONSTRAINTS: 100%|██████████| 8/8 [00:54<00:00,  6.81s/it]


6_41212417/08/202203:1323:5917/08/2022 + 6_43111317/08/202211:0023:5917/08/2022


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 110200 rows, 151794 columns and 323037 nonzeros
Model fingerprint: 0x50fabfe1
Model has 191140 general constraints
Variable types: 0 continuous, 151794 integer (112805 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
  GenCon rhs range [5e-01, 2e+01]
  GenCon coe range [1e+00, 2e+01]
Presolve removed 335156 rows and 381191 columns (presolve time = 5s) ...
Presolve removed 335156 rows and 381191 columns (presolve time = 10s) ...
Presolve removed 80508 rows and 139862 columns
Presolve time: 13.09s
Presolved: 29692 rows, 11932 columns, 90268 nonzeros
Variable types: 0 continuous, 11932 integer (11590 binary)
Found heuristic solution: 

Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [8]:
##### Respect du nombre de voies de chantier #####
def retrieve_occupations(OCCUPATIONS):
    # print(len(OCCUPATIONS))
    # print(OCCUPATIONS[0])
    # print(type(OCCUPATIONS[0]))
    # print(OCCUPATIONS[0].getValue())
    chantiers_df = dataframes["chantiers_df"]
    L = [[],[],[]]
    occupation_limits = [chantiers_df[chantiers_df["Chantier"]=="WPY_REC"]["Nombre de voies"].iloc[0],
            chantiers_df[chantiers_df["Chantier"]=="WPY_FOR"]["Nombre de voies"].iloc[0],
            chantiers_df[chantiers_df["Chantier"]=="WPY_DEP"]["Nombre de voies"].iloc[0]]
    for line_expr in OCCUPATIONS["DEB"]:
        v = line_expr.getValue()
        
        if v == occupation_limits[0]:
            print("DEB limite atteinte")
        if v > occupation_limits[0]:
            print("DEB limite DEPASSEE")
            
        L[0].append(v)
        
    for line_expr in OCCUPATIONS["FOR"]:
        v = line_expr.getValue()
        if v == occupation_limits[1]:
            print("FOR limite atteinte")
        if v > occupation_limits[1]:
            print("FOR limite DEPASSEE")
        L[1].append(v)


    for line_expr in OCCUPATIONS["DEG"]:
        v = line_expr.getValue()
        if v == occupation_limits[2]:
            print("DEG limite atteinte")
        if v > occupation_limits[2]:
            print("DEG limite DEPASSEE")
        L[2].append(v)
    return L,occupation_limits


In [12]:
show_solution(m,OCCUPATIONS)

Objectif minimal:  12645.0
DEG limite atteinte
DEG limite atteinte
DEG limite atteinte
DEG limite atteinte
DEG limite atteinte
DEG limite atteinte
DEG limite atteinte
DEG limite atteinte
DEG limite atteinte
